# Metamorphic Relation to calculate bias

## Visualisation using Tensorboard

In [1]:
#Importing model, training data set and validation data set from the CNN_classifier provided
from cnn_classifier import model, X_train, y_train, X_val, y_val

(39209, 30, 30, 3)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 31368 samples, validate on 7841 samples
31368/31368 [==============================] - 141s 5ms/sample - loss: 1.2049 - acc: 0.6631 - val_loss: 0.1435 - val_acc: 0.9621


In [3]:
import tensorflow as tf
import datetime

In [4]:
!rm -rf ./logs/

In [5]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [6]:
model.fit(x=X_train, 
          y=y_train, 
          epochs=2, 
          validation_data=(X_val, y_val), 
          callbacks=[tensorboard_callback])

Train on 31368 samples, validate on 7841 samples
Epoch 1/2
31368/31368 [==============================] - 295s 9ms/sample - loss: 0.2415 - acc: 0.9288 - val_loss: 0.0580 - val_acc: 0.9843
Epoch 2/2
31368/31368 [==============================] - 246s 8ms/sample - loss: 0.1604 - acc: 0.9510 - val_loss: 0.0468 - val_acc: 0.9856


In [7]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

# Detecting Bias and Fairness of our model using Google What-if tool

#### Help is taken from the notebook demo of WIT Smile Detector.ipynb to develop this code for my purpose

In [62]:
import numpy as np
import tensorflow as tf
import os
from PIL import Image
from io import BytesIO
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [63]:
def df_to_examples(df, columns=None, images_path=''):
    examples = []
    if columns == None:
        columns = df.columns.values.tolist()
    for index, row in df.iterrows():
        example = tf.train.Example()
    for col in columns:
        if df[col].dtype is np.dtype(np.int64):
            example.features.feature[col].int64_list.value.append(int(row[col]))
        elif df[col].dtype is np.dtype(np.float64):
            example.features.feature[col].float_list.value.append(row[col])
        elif row[col] == row[col]:
            example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
    if images_path:
        fname = row['Path']
        with open(os.path.join(images_path, fname), 'rb') as f:
            im = Image.open(f)
            buf = BytesIO()
            im.save(buf, format= 'PNG')
            im_bytes = buf.getvalue()
            example.features.feature['image/encoded'].bytes_list.value.append(im_bytes)
        examples.append(example)
    return examples

def make_label_column_numeric(df, label_column, test):
    df[label_column] = np.where(test(df[label_column]), 1, 0)

In [64]:
#@title Load the csv file into pandas dataframe and process it for WIT
import pandas as pd

data = pd.read_csv(r'/home/razorback/BTP/metamorphic_testing/traffic/gtsrb-german-traffic-sign/Train.csv')
data['image_id'] = range(1, len(data)+1)
path = r'/home/razorback/BTP/metamorphic_testing/traffic/gtsrb-german-traffic-sign/'
examples = df_to_examples(data)

In [65]:
def custom_predict(examples_to_infer):
    def load_byte_img(im_bytes):
        buf = BytesIO(im_bytes)
        return np.array(Image.open(buf), dtype=np.float64) / 255.

    ims = [load_byte_img(ex.features.feature['image/encoded'].bytes_list.value[0]) 
         for ex in examples_to_infer]
    preds = model.predict(np.array(ims))
    return preds


In [66]:
num_datapoints = 250  
tool_height_in_px = 700 

# Decode an image from tf.example bytestring
def decode_image(ex):
    im_bytes = ex.features.feature['image/encoded'].bytes_list.value[0]
    im = Image.open(BytesIO(im_bytes))
    return im

# Define the custom distance function that compares the average color of images
def image_mean_distance(ex, exs, params):
    selected_im = decode_image(ex)
    mean_color = np.mean(selected_im, axis=(0,1))
    image_distances = [np.linalg.norm(mean_color - np.mean(decode_image(e), axis=(0,1))) for e in exs]
    return image_distances

# Setup the tool with the test examples and the trained classifier
config_builder = WitConfigBuilder(examples[:num_datapoints]).set_custom_predict_fn(
    custom_predict).set_custom_distance_fn(image_mean_distance)

wv = WitWidget(config_builder, height=tool_height_in_px)

In [67]:
wv

WitWidget(config={'model_type': 'classification', 'label_vocab': [], 'inference_address': 'custom_predict_fn',…